In [1]:
import os
import sys
import time
import pandas as pd
import arcpy
from arcgis.features import GeoAccessor, GeoSeriesAccessor

In [2]:
# show all columns
pd.options.display.max_columns = None

In [3]:
# Read in hdf5 
store = pd.HDFStore('.\\Inputs\\remm_data_2015_base_year_11062020.h5')
tables = list(store.keys())
tables

['/buildings',
 '/buildings_for_estimation',
 '/buildings_for_estimation_grouped',
 '/households',
 '/households_for_estimation',
 '/jobs',
 '/parcels',
 '/valid_parcels',
 '/zoning',
 '/zoning_baseline',
 '/zoning_for_parcels']

In [7]:
# get the parcels table from the store
parcels = store.get('parcels').reset_index()
parcels

,parcel_id,CO_FIPS,zone_id,parcel_acres,land_value,x,y,elevation,fwy_exit,airport,rail_depot,stream,trail,university,shape_area,volume_one_way,volume_two_way,airport_distance,fwy_exit_dist,raildepot_dist,university_dist,trail_dist,stream_dist,train_station,rail_stn_dist,bus_rte_dist,bus_stop,bus_stop_dist,volume_two_way_nofwy,distsml_id,distmed_id,distlrg_id,parent_parcel,CO_NAME,parcel_id_REMM,county_taz_id,utmxi,utmyi,city,zonal_ppa,cid_old
0,741871,49,2111,0.197968,35700.00,1531239.186,7330722.151,1447.537450,0.0,0.0,0.0,1.0,0.0,0.0,8.623484e+03,992.0,1942.0,18.02655,0.85831,12.61234,14.73993,0.09764,0.00000,0.0,8.748286,2382.31,0.0,0.882466,1942.0,487,38,13,741871,UTAH,741871,492111,424331.7633,4477427.732,Lehi,176926.527483,4
1,579822,49,2873,37.717834,56614.32,1535437.548,7341595.932,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,1.642989e+06,3564.0,7135.0,19.48828,1.32984,10.63629,15.94851,0.23207,0.00000,0.0,6.691816,4930.50,0.0,1.164110,7135.0,0,0,0,579822,UTAH,579822,492873,425629.9856,4480733.739,Utah County,140869.334754,4
2,579853,49,2117,6.791235,81500.00,1534101.235,7334630.179,1657.284652,0.0,0.0,0.0,1.0,0.0,0.0,2.958262e+05,0.0,0.0,18.42054,1.66976,11.91035,15.00142,0.55081,0.00000,0.0,7.994607,6770.95,0.0,1.661999,0.0,487,38,13,579853,UTAH,579853,492117,425210.6619,4478613.554,Lehi,199096.575130,4
3,640185,49,2073,1.372797,112100.00,1538253.186,7322787.258,1445.761763,0.0,0.0,0.0,1.0,0.0,0.0,5.979902e+04,2495.0,5043.0,16.06753,0.60111,14.23267,12.73670,0.04966,0.00000,0.0,10.277766,2627.34,0.0,0.548186,5043.0,482,38,13,640185,UTAH,640185,492073,426455.1775,4474997.689,Lehi,169098.553773,4
4,682698,49,2119,0.163751,43000.00,1548648.398,7336198.043,1766.927829,0.0,0.0,0.0,1.0,0.0,0.0,7.132996e+03,3632.0,7133.0,17.61362,3.74880,12.20122,13.84603,0.37882,0.00000,0.0,8.210217,9997.32,0.0,2.129948,7133.0,488,39,14,682698,UTAH,682698,492119,429646.0053,4479065.962,Draper,254583.603528,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845214,886695,49,2666,4.531522,48006.00,1574258.679,7220957.846,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,1.973931e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886695,492666,437250.4157,4443906.735,Lake Shore,111196.498940,4
845215,886696,49,2666,3.158082,33456.00,1573684.703,7220955.159,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,1.375661e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886696,492666,437075.5120,4443906.904,Lake Shore,111196.498940,4
845216,886697,49,2666,4.895412,51862.00,1573301.933,7220958.410,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,2.132442e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886697,492666,436958.8819,4443908.552,Lake Shore,111196.498940,4
845217,886698,49,2666,4.918818,52109.00,1572835.592,7220958.477,1372.488928,0.0,0.0,0.0,0.0,0.0,0.0,2.142637e+05,107.0,214.0,5.32442,4.38052,6.67391,8.88569,3.24828,0.01905,0.0,30.544293,19622.91,0.0,5.026164,214.0,546,44,16,601655,UTAH,886698,492666,436816.7808,4443909.375,Lake Shore,111196.498940,4


In [22]:
redev_friction = parcels[['parcel_id']].copy()
redev_friction['redev_friction'] = 100000
redev_friction.head()
redev_friction.to_csv(r'.\\Outputs\\redev_friction.csv', index=False)

In [16]:
# get the parcels table from the store
zoning_baseline = store.get('zoning_baseline').reset_index()
zoning_baseline

,parcel_id,max_dua,max_far,type1,type2,type3,type4,type5,type6,type7,type8,max_height
0,671124,NaN,0.50,f,f,f,t,t,f,f,f,999
1,588354,NaN,0.50,f,f,f,t,t,f,f,f,999
2,664267,NaN,0.41,f,f,t,f,f,f,f,f,999
3,632453,4.000,NaN,t,f,f,f,f,f,f,f,999
4,632450,4.000,NaN,t,f,f,f,f,f,f,f,999
...,...,...,...,...,...,...,...,...,...,...,...,...
845214,886695,0.025,NaN,t,f,f,f,f,f,f,f,999
845215,886696,0.025,NaN,t,f,f,f,f,f,f,f,999
845216,886697,0.025,NaN,t,f,f,f,f,f,f,f,999
845217,886698,0.025,NaN,t,f,f,f,f,f,f,f,999


In [8]:
zoning_parcels_p = pd.read_csv(r"E:\Projects\REMM2_For_Python3\data\scenario_inputs\baseline\zoning_parcels_p.csv")
zoning_parcels_p

,parcel_id,max_dua,max_far,year,type1,type2,type3,type4,type5,type6,type7,type8,locnote,mponote
0,225355,0.0,0.50,2015,0,0,0,1,1,0,0,0,WFRCCenters,WFRC
1,77879,30.0,2.00,2015,0,1,0,1,1,0,0,0,WFRCCenters,WFRC
2,46945,0.0,0.41,2015,0,0,1,0,0,0,0,0,WFRCCenters,WFRC
3,456632,30.0,2.00,2015,0,1,0,1,1,0,0,0,WFRCCenters,WFRC
4,302359,10.0,0.80,2015,1,1,0,1,1,0,0,0,WFRCCenters,WFRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121198,882688,3.0,0.00,2045,1,0,0,0,0,0,0,0,Utah County,MAG
121199,882689,3.0,0.00,2045,1,0,0,0,0,0,0,0,Utah County,MAG
121200,882690,3.0,0.00,2045,1,0,0,0,0,0,0,0,Utah County,MAG
121201,882691,3.0,0.00,2045,1,0,0,0,0,0,0,0,Utah County,MAG


In [15]:
wfrccenters = zoning_parcels_p[zoning_parcels_p['locnote'] == 'WFRCCenters']
pd.DataFrame(wfrccenters['parcel_id'].value_counts()).to_csv(".\\Outputs\\parcel_ids.csv")